# Evaluating Bulk Variables

Some variables, such as relative fractions and wet depositions, are often fractions of other variables. The example we will be looking at here is the (nonsense) fraction of PM10 to O3: $PM10/O3$. Now imagine that we have daily observations, but want to look at the fractions on a monthly timescale. There are two ways of doing this, first is to read the daily observations, calculate the fraction then take the monthly mean of the fraction. This is way PyAerocom normally does all derived variables. But, there is a second way: First you do the monthly mean for the numerator and the denominator in the fraction separately, then take the fraction of the monthly means. This is different from the first method since $E[X/Y] \neq E[X]/E[Y]$. Variables where we want to use the latter method is called **Bulk Variables**

Since PyAerocom isn't well suited to handle bulk variables, PyAeroval has a special way of computing these, facilitated by the `bulkfraction_engine`. In this tutorial we will look at how to use this method in an evaluation to calculate the PM10 to O3 fraction.

## How Does it Work? (optional)

## Setting Up the Evaluation

Telling PyAeroval to use bulk variables are not difficult. First we have to download some test data we can use. If you haven't done so in another tutorial, run

In [7]:
!  pya getsampledata --extract-dir ../data

Extracting 'testdata-minimal' from '/home/danielh/MyPyaerocom/testdata-minimal.tar.gz.20241120' to '/lustre/storeB/project/fou/kl/emep/People/danielh/projects/pyaerocom/pyaerocom-tutorials/data'


We start making the config by setting up a normal EMEP model evaluation

In [ ]:
output_dir = <output_dir>"/data"
coldata_dir = <output_dir>"/coldata"
exp_pi = <your name>
proj_id = "tutorial"
exp_id = "bulk"

: 

In [ ]:
CFG = dict(
    # Output directories
    json_basedir=output_dir,
    coldata_basedir=coldata_dir,

    # Run options
    reanalyse_existing=True,        # if True, existing colocated data files will be deleted
    raise_exceptions=True,          # if True, the analysis will stop whenever an error occurs 
    clear_existing_json=False,      # if True, deletes previous output before running

    # Map Options
    add_model_maps=False,           # Adds a plot of the whole map. Very slow!!!
    only_model_maps=False,          # Adds only plot above, without any other evaluation
    filter_name="ALL-wMOUNTAINS",   # Regional filter for analysis
    map_zoom="Europe",              # Zoom level. For EMEP, Europe is typically used
    regions_how="country",          # Calculates statistics for different regions. Typically "country" is used, but that does not work for satellite data

    # Time and Frequency Options
    ts_type="monthly",              # Colocation frequency (no statistics in higher resolution can be computed)
    freqs=["monthly", "yearly"],    # Frequencies that are evaluated
    main_freq="monthly",            # Frequency that is displayed when opening webpage
    periods=["2019"],               # List of years or periods of years that are evaluated. E.g. "2005" or "2001-2020"
    

    # Statistical Options
    obs_remove_outliers=False,
    model_remove_outliers=False,
    colocate_time=True,
    zeros_to_nan=False,
    weighted_stats=True,
    annual_stats_constrained=True,
    harmonise_units=True,

    # Experiment Metadata
    exp_pi=exp_pi,
    proj_id=proj_id,
    exp_id=exp_id,
    exp_name="Evaluation of EMEP data with bulk variables",
    exp_descr=("Evaluation of EMEP data with bulk variables"),
    public=True,

    var_order_menu=["fraction"] # Adds variable to menu list, so it is visible at webpage 
)


In [ ]:
folder_EMEP = "../../data/testdata-minimal/modeldata/EMEP_PM"

EMEP = dict(
        model_id="EMEP",
        model_data_dir=folder_EMEP,
        gridded_reader_id={"model": "ReadMscwCtm"}, # Tells pyaerocom to use the EMEP reader instead of the default aerocom reader
    )

MODELS = {
    "EMEP": EMEP,
}

CFG["model_cfg"] = MODELS

## Adding a new Variable

We will be evaluating the variable `fraction`. This does not exist in PyAerocom, so we need to add it as a new variable. For more detail see our [Adding Custom Variables tutorial](https://pyaerocom.readthedocs.io/en/latest/pyaerocom-tutorials/evaluations/add_custom_variables.html)

In [ ]:
from pyaerocom.variable import Variable
from pyaerocom import const

variables = {
    "fraction": Variable(var_name="fraction", units="1"), # Defines a new variable
}

const.register_custom_variables(variables)

## Adding Bulk Variables to the Observation Network

We will now add an EBAS observations network using pyaro, with a few extra option, which we will look at afterwards

In [ ]:
from pyaerocom.io import PyaroConfig


data_id = "nilupmfebas"
url = "../../data/testdata-minimal/obsdata/EBASTutorialData"

config = PyaroConfig(
    name="ebaspyaro",
    reader_id=data_id,
    filename_or_obj_or_url=url,
    filters={
        "variables": {
            "include": [
                "pm10#pm10_mass#ug m-3",
                "pm25#pm25_mass#ug m-3",
            ]
        }
    },
    name_map={
        "pm25#pm25_mass#ug m-3": "concpm25",
        "pm10#pm10_mass#ug m-3": "concpm10",
    },
)

EBAS= dict(
            obs_id=config.name,
            pyaro_config=config,
            web_interface_name="EBAS-m",
            obs_vars=[
                "fraction"
            ],
            obs_vert_type="Surface",
            ts_type="monthly",
            is_bulk=True,
            bulk_options={
            "fraction": dict(
                vars=["concpm25", "concpm10"],
                model_exists=False,
                mode="fraction",
                units="1",
            ),}
            
        )

OBS_CFG = {
    "EBAS-m": EBAS,
}

CFG["obs_cfg"] = OBS_CFG

As we can see, there are two new options in the observation entry: `is_bulk` and `bulk_options`. 

`is_bulk`, as the name implies, is an option to tell pyAeroval that this observation entry is for bulk variables. Note that if this is true, then all the variables in `obs_vars` must be bulk variables.

`bulk_options` is where we define how to calculate our bulk variable. There are four options that needs to be defines:
- `vars`: This is the variables that are going to be combines. There must be exactly two variables in this list. For fraction, the order here is important!
- `mode`: This can be `fraction` or `product`, and defines how our original variables are combined
- `units`: The units of our resulting bulk variable
- `model_exist`: This tells the pyAeroval if our model has the bulk variable, in which case it will be read directly from the model. In our example, this is not the case, and the model variable will thus be calculated the same way as the observation variable, as the fraction `concpm10/conco3`

## Running the Evaluation

To run the evaluation, download the script [here](./scripts/bulk_variables.py) and run it.